In [11]:
import pandas as pd

# Пути к файлам и соответствующие названия колонок
file_paths = {
    # 'bert': '/bert/sim-words5-bert.csv',
    'glove': '/glove_python/sim-words_sent_5_glove.csv',
    'word2vec': '/word2vec/similar_words_sent_5.csv'
}


data = {}
for model_name, path in file_paths.items():
  full_path = ".." + path
  df = pd.read_csv(full_path)
  data[model_name] = df

In [12]:
import networkx as nx
from ast import literal_eval

graphs = {}
for model_name, df in data.items():
    G = nx.Graph()
    
    for _, row in df.iterrows():
        word = row['Word']
        similar_words = row['Most_Similar_Word']
        
        # Пропускаем пустые списки
        if not similar_words or pd.isna(similar_words):
            continue
        
        G.add_node(word)
        # Преобразуем строку в список кортежей (если данные в формате строки)
        if isinstance(similar_words, str):
            try:
                similar_words = literal_eval(similar_words)
            except (ValueError, SyntaxError):
                continue
        # Добавляем рёбра в граф
        for similar_word, weight in similar_words:
            G.add_edge(word, similar_word, weight=weight)
    graphs[model_name] = G    
    nx.write_gexf(G, f"{model_name}_graph.gexf")  # формат GEXF для Gephi
    print(f"Граф '{model_name}' содержит {len(G.nodes)} узлов и {len(G.edges)} рёбер.")

Граф 'glove' содержит 18402 узлов и 9921 рёбер.
Граф 'word2vec' содержит 18401 узлов и 86834 рёбер.


In [13]:
from community import community_louvain
for model_name, G in graphs.items():
  print(f"model name: {model_name}")
  partition = community_louvain.best_partition(G, weight='weight')
  # Кол-во кластеров
  num_clusters = max(partition.values()) + 1
  print(f"Число кластеров (Louvain): {num_clusters}")
  
  # Размеры кластеров
  from collections import Counter
  cluster_sizes = Counter(partition.values())
  print(f"Размеры кластеров: {cluster_sizes.most_common(5)}")  # Топ-5 кластеров

  # 3. Модулярность (качество кластеризации)
  modularity = community_louvain.modularity(partition, G, weight='weight')
  print(f"Модулярность: {modularity:.3f}")

model name: glove
Число кластеров (Louvain): 15503
Размеры кластеров: [(259, 425), (54, 374), (6827, 248), (258, 214), (329, 214)]
Модулярность: 0.460
model name: word2vec
Число кластеров (Louvain): 5854
Размеры кластеров: [(36, 1666), (14, 1543), (65, 1315), (84, 1236), (19, 1103)]
Модулярность: 0.625
